# Data Preprocessing

This notebook clarifies the nature of our data and the series of preprocessing steps implemented. We've adopted the Medallion Architecture as our guiding principle for data refinement. Within this structure, every data source progresses through three distinct layers: Bronze, Silver, and Gold. Each layer serves its own unique purpose in the pipeline.

## Overview of the Layers:

### 1. **Bronze Layer (Raw Data)**
- **Nature**: This is the landing area for our raw data, ingested directly from the source without any alterations.
- **Purpose**: To store an immutable, 1:1 replica of the source data. It serves as the foundational bedrock upon which further layers are built.

### 2. **Silver Layer (Cleaned Data)**
- **Nature**: Data in this layer has been cleaned, enriched, and is stored in a format suitable for analysis. Any inconsistencies, missing values, or anomalies from the Bronze layer have been addressed here.
- **Purpose**: To have a reliable, single version of the truth which is suitable for analysis but without any specific business logic applied. This is the primary layer for data scientists and analysts to query against.

### 3. **Gold Layer (Business-Ready Data)**
- **Nature**: This layer houses data that has been aggregated, enriched, and optimized for specific business use-cases. It is derived from the Silver layer.
- **Purpose**: To provide business-ready datasets for driving insights, reports, visualizations, and machine learning models. This layer is tailored to end-users and specific analytical objectives.


In [14]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import seaborn as sns
from scipy.stats import normaltest


### Player skill dataset

In [47]:
# Bronze Layer

bronze_player_ratings = pd.read_csv('../data/FUT_player_data.csv')
pd.set_option('display.max_columns', None)
bronze_player_ratings.head()

,id,futbin_id,name,height,weight,age,club,league,nation,rarity,position,foot,attackWorkRate,defenseWorkRate,cardColor,overallRating,pace,shooting,passing,dribbling,defending,physicality,pace_acceleration,pace_sprintSpeed,shooting_positioning,shooting_finishing,shooting_shotPower,shooting_longShots,shooting_volleys,shooting_penalties,passing_vision,passing_crossing,passing_freeKickAccuracy,passing_shortPassing,passing_longPassing,passing_curve,dribbling_agility,dribbling_balance,dribbling_reactions,dribbling_ballControl,dribbling_dribbling,dribbling_composure,defending_interceptions,defending_headingAccuracy,defending_standingTackle,defending_slidingTackle,defending_defenseAwareness,phsyicality_jumping,physicality_stamina,physicality_strength,physicality_aggression,goalkeeper_diving,goalkeeper_handling,goalkeeper_kicking,goalkeeper_positioning,goalkeeper_reflexes,goalkeeper_speed
0,18949,54231,Kylian Mbappé,182,73,24,73,16,18,16,ST,Right,High,Low,gold,99,99,98,92,99,45,87,99.0,99.0,99.0,99.0,99.0,94.0,95.0,93.0,95.0,90.0,80.0,97.0,82.0,92.0,99.0,91.0,99.0,99.0,99.0,99.0,48.0,87.0,43.0,40.0,33.0,88.0,99.0,87.0,73.0,NaN,NaN,NaN,NaN,NaN,99
1,18981,54251,Karim Benzema,185,81,35,607,350,18,164,CF,Right,High,Med,gold,99,97,99,93,98,50,97,97.0,97.0,99.0,99.0,99.0,97.0,99.0,97.0,99.0,84.0,83.0,98.0,86.0,93.0,91.0,90.0,99.0,99.0,99.0,99.0,55.0,99.0,33.0,25.0,56.0,99.0,99.0,99.0,90.0,NaN,NaN,NaN,NaN,NaN,97
2,18982,54249,Zinedine Zidane,185,77,51,112658,2118,18,171,CAM,Right,Med,Med,gold,99,92,96,99,97,87,90,93.0,92.0,96.0,95.0,95.0,99.0,97.0,94.0,99.0,99.0,99.0,99.0,99.0,99.0,88.0,90.0,99.0,99.0,99.0,99.0,95.0,99.0,88.0,72.0,83.0,87.0,94.0,92.0,83.0,NaN,NaN,NaN,NaN,NaN,92
3,18730,54005,Pelé,173,70,82,112658,2118,54,153,LW,Right,High,Med,gold,99,96,97,94,99,61,78,96.0,96.0,98.0,99.0,95.0,95.0,96.0,94.0,98.0,91.0,90.0,97.0,89.0,90.0,97.0,96.0,99.0,99.0,99.0,99.0,68.0,96.0,54.0,50.0,56.0,90.0,91.0,78.0,61.0,NaN,NaN,NaN,NaN,NaN,96
4,19001,54277,Robert Lewandowski,185,81,35,241,53,37,164,ST,Right,High,Med,gold,99,97,99,92,97,53,99,98.0,97.0,99.0,99.0,99.0,99.0,99.0,99.0,94.0,83.0,99.0,98.0,82.0,92.0,87.0,92.0,99.0,99.0,97.0,99.0,60.0,99.0,52.0,24.0,42.0,99.0,97.0,99.0,99.0,NaN,NaN,NaN,NaN,NaN,97


In [ ]:
# List of numerical columns
numerical_cols = [
    'height', 'weight', 'age', 'overallRating', 'pace', 'shooting',
    'passing', 'dribbling', 'defending', 'physicality',
    'pace_acceleration', 'pace_sprintSpeed', 'shooting_positioning',
    'shooting_finishing', 'shooting_shotPower', 'shooting_longShots',
    'shooting_volleys', 'shooting_penalties', 'passing_vision',
    'passing_crossing', 'passing_freeKickAccuracy', 'passing_shortPassing',
    'passing_longPassing', 'passing_curve', 'dribbling_agility',
    'dribbling_balance', 'dribbling_reactions', 'dribbling_ballControl',
    'dribbling_dribbling', 'dribbling_composure', 'defending_interceptions',
    'defending_headingAccuracy', 'defending_standingTackle',
    'defending_slidingTackle', 'defending_defenseAwareness',
    'phsyicality_jumping', 'physicality_stamina', 'physicality_strength',
    'physicality_aggression', 'goalkeeper_diving', 'goalkeeper_handling',
    'goalkeeper_kicking', 'goalkeeper_positioning', 'goalkeeper_reflexes',
    'goalkeeper_speed'
]

# List of categorical columns
categorical_cols = [
     'position', 'foot',
    'attackWorkRate', 'defenseWorkRate', 'cardColor'
]

In [51]:
# Silver Layer
silver_player_ratings = bronze_player_ratings.copy()
# substitute the foreign keys with the actual values 

#read csv files as dicts
club_ids = pd.read_csv('../data/club_ids.csv', index_col = 'id')
league_ids = pd.read_csv('../data/league_ids.csv', index_col = 'id')
nation_ids = pd.read_csv('../data/nation_ids.csv', index_col = 'id')

club_ids =  {k:v[0] for k, v in zip(club_ids.index, club_ids.values)}
league_ids = {k:v[0] for k, v in zip(league_ids.index, league_ids.values)}
nation_ids = {k:v[0] for k, v in zip(nation_ids.index, nation_ids.values)}

# this is a master dictionary 
master_dict = {'club': club_ids, 'league': league_ids, 'nation': nation_ids}

# replace the values 
silver_player_ratings = silver_player_ratings.replace(master_dict)

# Handle missing values for numerical columns with mean of that column
for col in numerical_cols:
    silver_player_ratings[col].fillna(silver_player_ratings[col].mean(), inplace=True)

# Handle missing values for categorical columns with mode (most frequent value) of that column
for col in categorical_cols:
    silver_player_ratings[col].fillna(silver_player_ratings[col].mode()[0], inplace=True)

# Take out Icon samples
silver_player_ratings = silver_player_ratings[silver_player_ratings['league'] != 'Icons']
silver_player_ratings

,id,futbin_id,name,height,weight,age,club,league,nation,rarity,position,foot,attackWorkRate,defenseWorkRate,cardColor,overallRating,pace,shooting,passing,dribbling,defending,physicality,pace_acceleration,pace_sprintSpeed,shooting_positioning,shooting_finishing,shooting_shotPower,shooting_longShots,shooting_volleys,shooting_penalties,passing_vision,passing_crossing,passing_freeKickAccuracy,passing_shortPassing,passing_longPassing,passing_curve,dribbling_agility,dribbling_balance,dribbling_reactions,dribbling_ballControl,dribbling_dribbling,dribbling_composure,defending_interceptions,defending_headingAccuracy,defending_standingTackle,defending_slidingTackle,defending_defenseAwareness,phsyicality_jumping,physicality_stamina,physicality_strength,physicality_aggression,goalkeeper_diving,goalkeeper_handling,goalkeeper_kicking,goalkeeper_positioning,goalkeeper_reflexes,goalkeeper_speed
0,18949,54231,Kylian Mbappé,182,73,24,Paris SG,Ligue 1,France,16,ST,Right,High,Low,gold,99,99,98,92,99,45,87,99.000000,99.000000,99.000000,99.000000,99.000000,94.000000,95.000000,93.000000,95.000000,90.000000,80.000000,97.000000,82.000000,92.000000,99.00000,91.000000,99.00000,99.000000,99.000000,99.0,48.000000,87.000000,43.000000,40.000000,33.000000,88.000000,99.000000,87.000000,73.000000,67.048519,64.859551,63.755363,65.376404,68.046476,99
1,18981,54251,Karim Benzema,185,81,35,Al Ittihad,MBS Pro League (SAU 1),France,164,CF,Right,High,Med,gold,99,97,99,93,98,50,97,97.000000,97.000000,99.000000,99.000000,99.000000,97.000000,99.000000,97.000000,99.000000,84.000000,83.000000,98.000000,86.000000,93.000000,91.00000,90.000000,99.00000,99.000000,99.000000,99.0,55.000000,99.000000,33.000000,25.000000,56.000000,99.000000,99.000000,99.000000,90.000000,67.048519,64.859551,63.755363,65.376404,68.046476,97
4,19001,54277,Robert Lewandowski,185,81,35,FC Barcelona,LaLiga Santander,Poland,164,ST,Right,High,Med,gold,99,97,99,92,97,53,99,98.000000,97.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,94.000000,83.000000,99.000000,98.000000,82.000000,92.000000,87.00000,92.000000,99.00000,99.000000,97.000000,99.0,60.000000,99.000000,52.000000,24.000000,42.000000,99.000000,97.000000,99.000000,99.000000,67.048519,64.859551,63.755363,65.376404,68.046476,97
5,19003,54275,Erling Haaland,195,94,23,Manchester City,Premier League,Norway,164,ST,Left,High,Med,gold,99,99,99,88,94,63,99,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,99.000000,90.000000,96.000000,62.000000,80.000000,96.000000,90.000000,99.000000,91.00000,92.000000,99.00000,97.000000,92.000000,99.0,56.000000,99.000000,69.000000,38.000000,57.000000,91.000000,99.000000,99.000000,99.000000,67.048519,64.859551,63.755363,65.376404,68.046476,99
6,19004,54274,Gianluigi Donnarumma,196,90,24,Paris SG,Ligue 1,Italy,164,GK,Right,Med,High,gold,99,0,0,0,0,0,0,68.066656,68.147454,55.533568,51.065535,61.969581,51.730281,47.307503,51.847422,58.530099,53.992422,47.120931,63.440068,57.733643,52.584587,66.94786,66.941669,65.43169,63.142011,60.557904,65.0,49.955171,55.798484,51.261874,48.711869,49.854147,67.265183,67.075675,67.807237,59.355534,99.000000,95.000000,92.000000,98.000000,99.000000,77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19030,14789,47568,Jiahui Zhang,182,0,20,Hebei CFFC,Chinese FA Super L. (CHN 1),China PR,0,CM,Right,Med,Med,bronze,46,54,37,46,46,43,51,48.000000,58.000000,52.000000,30.000000,51.000000,33.000000,36.000000,42.000000,44.000000,38.000000,35.000000,53.000000,50.000000,33.000000,61.00000,62.000000,41.00000,45.000000,42.000000,43.0,40.000000,49.000000,45.000000,46.000000,40.000000,51.000000,43.000000,54.000000,51.000000,67.048519,64.859551,63.755363,65.376404,68.046476,53
19031,14790,47577,Ziye Zhao,180,0,19,Hebei CFFC,Chinese FA Super L. (CHN 1),China PR,0,RW,Right,Med,Med,bronze,46,63,48,40,47,2

In [31]:
# Gold Layer
player_ratings_scaled = silver_player_ratings.copy()


# Standardize numerical columns
scaler = StandardScaler()
player_ratings_scaled[numerical_cols] = scaler.fit_transform(player_ratings_scaled[numerical_cols])

# One-hot encode categorical columns
player_ratings_scaled = pd.get_dummies(player_ratings_scaled, columns=categorical_cols)
player_ratings_scaled

,id,futbin_id,name,height,weight,age,club,league,nation,rarity,overallRating,pace,shooting,passing,dribbling,defending,physicality,pace_acceleration,pace_sprintSpeed,shooting_positioning,shooting_finishing,shooting_shotPower,shooting_longShots,shooting_volleys,shooting_penalties,passing_vision,passing_crossing,passing_freeKickAccuracy,passing_shortPassing,passing_longPassing,passing_curve,dribbling_agility,dribbling_balance,dribbling_reactions,dribbling_ballControl,dribbling_dribbling,dribbling_composure,defending_interceptions,defending_headingAccuracy,defending_standingTackle,defending_slidingTackle,defending_defenseAwareness,phsyicality_jumping,physicality_stamina,physicality_strength,physicality_aggression,goalkeeper_diving,goalkeeper_handling,goalkeeper_kicking,goalkeeper_positioning,goalkeeper_reflexes,goalkeeper_speed,position_CAM,position_CB,position_CDM,position_CF,position_CM,position_GK,position_LB,position_LM,position_LW,position_LWB,position_RB,position_RM,position_RW,position_RWB,position_ST,foot_Left,foot_Right,attackWorkRate_High,attackWorkRate_Low,attackWorkRate_Med,defenseWorkRate_High,defenseWorkRate_Low,defenseWorkRate_Med,cardColor_bronze,cardColor_gold,cardColor_silver
0,18949,54231,Kylian Mbappé,0.069935,1.521727,-0.441774,73,16,18,16,3.006927,1.798925,2.238685,1.951703,2.170112,-0.236559,1.480001,1.938032,1.951400,2.042407,2.227204,2.423348,1.996066,2.370765,2.334378,2.294032,1.859971,1.727380,2.119865,1.499414,1.969703,2.044723,1.575250,2.819737,2.013257,1.950297,2.488115,-0.091204,1.683354,-0.379519,-0.407687,-0.807338,1.558578,1.887697,1.453339,0.774599,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.972583,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,1,0
1,18981,54251,Karim Benzema,0.346944,1.756129,0.756160,607,350,18,164,3.006927,1.686473,2.287935,2.007656,2.110459,0.001856,2.141186,1.812729,1.824902,2.042407,2.227204,2.423348,2.137732,2.569602,2.561278,2.545640,1.550041,1.884993,2.183031,1.746574,2.019676,1.534374,1.509774,2.819737,2.013257,1.950297,2.488115,0.235329,2.330767,-0.838881,-1.109638,0.294395,2.385417,1.887697,2.362018,1.739693,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.847251,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0
2,18982,54249,Zinedine Zidane,0.346944,1.638928,2.498610,112658,2118,18,171,3.006927,1.405342,2.140185,2.343375,2.050807,1.766128,1.678356,1.562121,1.508655,1.901442,2.041350,2.161580,2.232176,2.470184,2.391103,2.545640,2.324865,2.725591,2.246198,2.549841,2.319514,1.342993,1.509774,2.819737,2.013257,1.950297,2.488115,2.101233,2.330767,1.687609,1.089808,1.587735,1.483411,1.592045,1.831955,1.342301,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.533922,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,1,0
3,18730,54005,Pelé,-0.761091,1.433825,5.874606,112658,2118,54,153,3.006927,1.630247,2.189435,2.063609,2.170112,0.526369,0.884935,1.750077,1.761652,1.995418,2.227204,2.161580,2.043288,2.420474,2.391103,2.482738,1.911626,2.252754,2.119865,1.931943,1.869758,1.917136,1.902631,2.819737,2.013257,1.950297,2.488115,0.841748,2.168914,0.125779,0.060280,0.294395,1.708912,1.414654,0.771829,0.093357,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.784585,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0
4,19001,54277,Robert Lewandowski,0.346944,1.756129,0.756160,241,53,37,164,3.006927,1.686473,2.287935,1.951703,2.050807,0.144905,2.273422,1.875380,1.824902,2.042407,2.227204,2.423348,2.232176,2.569602,2.674728,2.231130,1.498386,2.725591,2.183031,1.499414,1.969703,1.279199,1.640726,2.819737,2.013257,1.848830,2.488115,0.468567,2.330767,0.033907,-1.156435,-0.376225,2.385417,1.769436,2.362018,2.250625,4.773320e-15,0.000000,-9.959911e-15,-4.450662e-15,-4.576791e-15,1.847251,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

### Transfer Fees

In [12]:
# Bronze Layer

top_5_leagues = [
    'combined_premier-league.csv', 
    'combined_serie-a.csv', 
    'combined_laliga.csv', 
    'combined_1-bundesliga.csv', 
    'combined_ligue-1.csv'
    ]

dfs  = []
for league in top_5_leagues:
    dfs.append(pd.read_csv(f'../data/{league}'))

bronze_transfer_fees = pd.concat(dfs, ignore_index=True)
print(bronze_transfer_fees.columns)
bronze_transfer_fees

Index(['club', 'name', 'age', 'nationality', 'position', 'short_pos',
       'market_value', 'dealing_club', 'dealing_country', 'fee', 'movement',
       'window', 'league', 'season', 'is_loan', 'loan_status', 'Year'],
      dtype='object')


,club,name,age,nationality,position,short_pos,market_value,dealing_club,dealing_country,fee,movement,window,league,season,is_loan,loan_status,Year
0,Arsenal FC,John Jensen,27.0,Denmark,Central Midfield,CM,NaN,Bröndby IF,Denmark,1600000.0,in,summer,Premier League,1992,False,NaN,1992
1,Arsenal FC,David Rocastle,25.0,England,Attacking Midfield,AM,NaN,Leeds,England,2400000.0,out,summer,Premier League,1992,False,NaN,1992
2,Arsenal FC,Perry Groves,27.0,England,Left Winger,LW,NaN,Southampton,England,850000.0,out,summer,Premier League,1992,False,NaN,1992
3,Arsenal FC,Gary McKeown,21.0,England,midfield,midfield,NaN,Dundee FC,Scotland,NaN,out,summer,Premier League,1992,False,NaN,1992
4,Arsenal FC,Ty Gooden,19.0,England,midfield,midfield,NaN,Wycombe,England,NaN,out,summer,Premier League,1992,False,NaN,1992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77091,Stade Rennais FC,Jérémy Gélin,23.0,France,Centre-Back,CB,4800000.0,Royal Antwerp,Belgium,0.0,out,summer,Ligue 1,2020,True,free loan,2020
77092,Stade Rennais FC,Riffi Mandanda,27.0,DR Congo,Goalkeeper,GK,325000.0,Without Club,NaN,NaN,out,summer,Ligue 1,2020,False,NaN,2020
77093,Stade Rennais FC,Joris Gnagnon,23.0,France,Centre-Back,CB,4800000.0,Sevilla FC,Spain,0.0,out,summer,Ligue 1,2020,True,end of loan,2020
77094,Stade Rennais FC,M'Baye Niang,26.0,Senegal,Centre-Forward,CF,10000000.0,Ahli,Saudi Arabia,1500000.0,out,winter,Ligue 1,2020,True,loan with fee,2020


In [53]:
# Silver Layer
silver_transfer_fees = bronze_transfer_fees.copy()

# Fill NaN values
silver_transfer_fees['market_value'].fillna(0, inplace=True)
silver_transfer_fees['dealing_club'].fillna("Unknown", inplace=True)
silver_transfer_fees['dealing_country'].fillna("Unknown", inplace=True)
silver_transfer_fees['fee'].fillna(0, inplace=True)
silver_transfer_fees['movement'].fillna("Unknown", inplace=True)
silver_transfer_fees['window'].fillna("Unknown", inplace=True)
silver_transfer_fees['loan_status'].fillna("Not Applicable", inplace=True)

# Standardize String Values
silver_transfer_fees['club'] = silver_transfer_fees['club'].str.title()
silver_transfer_fees['name'] = silver_transfer_fees['name'].str.title()
silver_transfer_fees['position'] = silver_transfer_fees['position'].str.title()
silver_transfer_fees['short_pos'] = silver_transfer_fees['short_pos'].str.upper()
silver_transfer_fees['nationality'] = silver_transfer_fees['nationality'].str.title()
silver_transfer_fees['dealing_club'] = silver_transfer_fees['dealing_club'].str.title()
silver_transfer_fees['dealing_country'] = silver_transfer_fees['dealing_country'].str.title()
silver_transfer_fees['league'] = silver_transfer_fees['league'].str.title()

# Feature Engineering: Derive Age Group
silver_transfer_fees['age_group'] = pd.cut(silver_transfer_fees['age'], bins=[0, 20, 25, 30, 100], labels=['<20', '20-25', '25-30', '30+'])

# Handle samples that have 0 transfer fee (joined another club for free (contract ending))
silver_transfer_fees = silver_transfer_fees[silver_transfer_fees['fee'] != 0]
silver_transfer_fees

,club,name,age,nationality,position,short_pos,market_value,dealing_club,dealing_country,fee,movement,window,league,season,is_loan,loan_status,Year,age_group
13630,Afc Bournemouth,Jordon Ibe,20.0,England,Right Winger,RW,7000000.0,Liverpool,England,18000000.0,in,summer,Premier League,2016,False,Not Applicable,2016,<20
13631,Afc Bournemouth,Lewis Cook,19.0,England,Central Midfield,CM,4000000.0,Leeds,England,7000000.0,in,summer,Premier League,2016,False,Not Applicable,2016,<20
13632,Afc Bournemouth,Lys Mousset,20.0,France,Centre-Forward,CF,400000.0,Ac Le Havre,France,6500000.0,in,summer,Premier League,2016,False,Not Applicable,2016,<20
13633,Afc Bournemouth,Brad Smith,22.0,Australia,Left-Back,LB,100000.0,Liverpool,England,3600000.0,in,summer,Premier League,2016,False,Not Applicable,2016,20-25
13634,Afc Bournemouth,Jack Wilshere,24.0,England,Central Midfield,CM,23000000.0,Arsenal,England,2350000.0,in,summer,Premier League,2016,True,loan with fee,2016,20-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77079,Stade Rennais Fc,Souleyman Doumbia,23.0,Cote D'Ivoire,Left-Back,LB,1600000.0,Sco Angers,France,3000000.0,out,summer,Ligue 1,2020,False,Not Applicable,2020,20-25
77080,Stade Rennais Fc,Lucas Da Cunha,19.0,France,Left Winger,LW,1800000.0,Ogc Nice,France,1000000.0,out,summer,Ligue 1,2020,False,Not Applicable,2020,<20
77081,Stade Rennais Fc,Denis Will Poha,23.0,France,Central Midfield,CM,1200000.0,Vit. Guimarães,Portugal,300000.0,out,summer,Ligue 1,2020,False,Not Applicable,2020,20-25
77094,Stade Rennais Fc,M'Baye Niang,26.0,Senegal,Centre-Forward,CF,10000000.0,Ahli,Saudi Arabia,1500000.0,out,winter,Ligue 1,2020,True,loan with fee,2020,25-30
